In [1]:
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

In [9]:
modelname = f'models/vgg19b/tl-vgg19_model_12.h5'

In [10]:
model = load_model(modelname)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 645   

In [12]:
w, h = (224, 224)
test_generator = ImageDataGenerator()

fdata = f'../../data/sars-cov2-ct-scan'

p = Path(fdata)
ptest = p / 'test'

In [13]:
testgen = test_generator.flow_from_directory(ptest,
                                             target_size=(w, h),
                                             class_mode='binary',
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 495 images belonging to 2 classes.


In [14]:
testgen.class_indices

{'COVID': 0, 'non-COVID': 1}

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

num_of_test_samples = len(testgen.classes)
batch_size = 1
Y_pred = model.predict_generator(testgen, num_of_test_samples // batch_size)
y_pred = [round(i[0]) for i in Y_pred]
print('Confusion Matrix')
print(confusion_matrix(testgen.classes, y_pred))
print('Classification Report')
target_names = ['COVID', 'Non-COVID']
print(classification_report(testgen.classes, y_pred, target_names=target_names))

C:\Users\ankan\AppData\Local\Temp\ipykernel_2372\2839527870.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(testgen, num_of_test_samples // batch_size)


Confusion Matrix
[[137 113]
 [ 80 165]]
Classification Report
              precision    recall  f1-score   support

       COVID       0.63      0.55      0.59       250
   Non-COVID       0.59      0.67      0.63       245

    accuracy                           0.61       495
   macro avg       0.61      0.61      0.61       495
weighted avg       0.61      0.61      0.61       495

